In [1]:
from services.dbconnection import DbService 
dbservice = DbService()



init


INSERT / UPDATE SERVICE DEFINITION

In [3]:
### insert service definition example ###
info = {
    "title": "skin treatments",
    "description" :"skin treatments",
    "ar_title" : "علاجات الجلد",
    "ar_description": "علاجات الجلد",
    "keywords": ["skin", "laser skin","skin treatments","facial"],
    "ar_keywords": ["تفتيح البشرة", "جلد","جلدية","فايشال"] 
}


dbservice.insert_service_definition(3,"علاجات البشرة",True, info)
#dbservice.update_service_definition(3,"skin treatments",True, info)

LIST MAIN SERVICES (SERVICE DEFINITION)

In [ ]:
from dbconnection import DbService 
dbservice = DbService()

dbservice.list_main_services(2)

INSERT / UPDATE SERVICE 

In [ ]:
### insert / update service  example ###
from dbconnection import DbService 
dbservice = DbService()

info = {
    "title": "skin treatments",
    "description" :"skin treatments",
    "ar_title" : "حب الشباب",
    "ar_description": "علاجات حب الشباب",
    "keywords": ["acne", "dark circles"],
    "ar_keywords": ["حب شباب","هالات سوداء"] 

}
#dbservice.insert_service(3,"dark spots",True, info)
dbservice.update_service(2,"dark spots",True, info)

UPDATE INSERT WORKING DAYS & HRS 

In [ ]:
### insert / update service  example ###
from dbconnection import DbService 
dbservice = DbService()

info = {
   "days": [
     
     {"title" : "Sunday", "timings": [{"start":"08:00", "end":"13:00"}, {"start":"15:00", "end":"20:30"}] },
     {"title" : "Monday", "timings": [{"start":"08:00", "end":"13:00"}, {"start":"15:00", "end":"20:30"}] },
     {"title" : "Tuesday", "timings": [{"start":"08:00", "end":"13:00"}, {"start":"15:00", "end":"20:30"}] },
     {"title" : "Wednesday", "timings": [{"start":"08:00", "end":"13:00"}, {"start":"15:00", "end":"20:30"}] },
     {"title" : "Thursday", "timings": [{"start":"08:00", "end":"13:00"}, {"start":"15:00", "end":"20:30"}] },
     {"title" : "Saturday", "timings": [{"start":"12:00", "end":"16:00"}, {"start":"15:00", "end":"20:30"}] }
   ] 

}
#dbservice.insert_service(3,"dark spots",True, info)
dbservice.insert_service_working_days(3, info)
#dbservice.update_service_working_days(3, info)
results = dbservice.list_service_working_days(3)
print(results)

ADD EXCEPTION HRS (NOT AVAILABLE)

In [ ]:
from datetimerange import DateTimeRange
from datetime import datetime
excep_from_date = datetime.strptime("25-11-2022 11:30",'%d-%m-%Y %H:%M')
excep_to_date = datetime.strptime("25-11-2022 12:30", '%d-%m-%Y %H:%M')

#dbservice.insert_working_hrs_exception(3,excep_from_date,excep_to_date)


INSERT APPOINTMENT 

In [ ]:

import datetime as dt
from datetime import datetime
from datetime import time
from errno import ETIME
service_def_id = 3
date_of_interest = datetime.strptime("25-11-2022 09:30:00",'%d-%m-%Y %H:%M:%S')
## service_def_id , name, phoe, datetime, status (1 for new)
dbservice.insert_appointment(3,"James K","12345555",date_of_interest,1)

LIST VALID APPOINTMENTS BY FROM DATE AND PHONE

In [ ]:
import datetime as dt
from datetime import datetime
from datetime import time
from dbconnection import DbService 
dbservice = DbService()

from_date = datetime.strptime("25-11-2022 12:30", '%d-%m-%Y %H:%M')

result = dbservice.list_valid_appointments_by_phone(3,from_date,"+96812345678")
print(result)

LIST VALID APPOINTMENTS BY DATE

In [ ]:
apps = dbservice.list_valid_appointments_by_phone(3,excep_from_date)

GET AVAILABLE SLOTS FOR A SPECIFIC DATE

CALL get_available_slots_by_date(date_of_interest, service_def_id)

In [ ]:
from dbconnection import DbService 
dbservice = DbService()
from datetimerange import DateTimeRange
import datetime as dt
from datetime import datetime
from datetime import time
from errno import ETIME
DURATION = 30
def getNameOfWeek(n):
    if n == 0:
        return 'Monday'
    if n == 1:
        return 'Tuesday'
    if n == 2:
        return 'Wednesday'
    if n == 3:
        return 'Thursday'
    if n == 4:
        return 'Thursday'
    if n == 5:
        return 'Saturday'
    if n == 6:
        return 'Sunday'

def get_valid_slots_by_day(start_apt_date, days_of_week):
    day_name = getNameOfWeek(start_apt_date.weekday()).lower()
    print(day_name)
    for item in days_of_week:
        print(item)
        if item['title'].lower() == day_name:
            final_slots = []
            for timerange in item['timings']:
                start_date_time = datetime.combine(start_apt_date.date(), time.fromisoformat(timerange['start']))
                end_date_time = datetime.combine(start_apt_date.date(), time.fromisoformat(timerange['end']))
                while start_date_time + dt.timedelta(minutes=DURATION) <= end_date_time:
                      final_slots.append(start_date_time)
                      start_date_time = start_date_time + dt.timedelta(minutes=DURATION)

            return final_slots

    return []

def get_available_slots(date_of_interest,appointments, working_days, exception_dates, duration=30):
    
    available_slots = get_valid_slots_by_day(date_of_interest,working_days['info']['days'])
    final_slots = []
    if len(available_slots) > 0:
        for slot in available_slots:
            print("checking slot " + str(slot))
            available_slot_start_time = slot + dt.timedelta(minutes=1)
            available_slot_end_time = slot + dt.timedelta(minutes=DURATION - 1)
            time_range1 = DateTimeRange(available_slot_start_time, available_slot_end_time)
            is_available = True
            for appointment in appointments:
                    start_time = appointment['appointment_date'] 
                    end_time =   appointment['appointment_date'] + dt.timedelta(minutes=DURATION)
                    time_range2 = DateTimeRange(start_time,end_time) 
                    if time_range1.intersection(time_range2).start_datetime != None:
                       is_available = False
                       print('conflict with ' + str(time_range2))
                       break
                    for exc_date in exception_dates:
                        start_time = exc_date['exception_from_date'] 
                        end_time =   exc_date['exception_to_date'] 
                        time_range2 = DateTimeRange(start_time,end_time) 
                        if time_range1.intersection(time_range2).start_datetime != None:
                            is_available = False
                            break
            if is_available:
                final_slots.append(slot)
                
            else: 
                print('not available')

    return final_slots

def get_available_slots_by_date(date_of_interest, service_def_id):
    appointments = dbservice.list_valid_appointments_by_date(3,date_of_interest)
    exception_dates = dbservice.list_service_working_hrs_exception_by_date(service_def_id, date_of_interest)
    working_days = dbservice.list_service_working_days(service_def_id)
    slots = get_available_slots(date_of_interest,appointments, working_days, exception_dates, duration=30)
    return slots


date_of_interest = datetime.strptime("25-11-2022 00:00:00",'%d-%m-%Y %H:%M:%S')

slots = get_available_slots_by_date(date_of_interest, 3)

INSERT SUGGESTION 

In [4]:
from dbconnection import DbService 
dbservice = DbService()

dbservice.insert_suggestion(3,"nadir","123456","Can you support additional languages")

init


1

LIST SUGGESTIONS

In [1]:
from dbconnection import DbService 
dbservice = DbService()

dbservice.list_suggestions(3)

init


[{'id': 1,
  'company_id': 3,
  'customer_name': 'nadir',
  'customer_phone': '123456',
  'suggestion_text': 'Can you support additional languages',
  'date_received': datetime.datetime(2022, 11, 8, 23, 49)}]

INSERT PROMOTION

In [2]:
from dbconnection import DbService 
dbservice = DbService()

dbservice.insert_promotion(3,"تخفيضات العيد","تخفيضات تصل الى 40% على جميع الخدمات لغاية نهاية الشهر",1)

init


1

LIST PROMOTION

In [3]:
dbservice.list_valid_promotions(3)

[{'id': 1,
  'company_id': 3,
  'title': 'تخفيضات العيد',
  'promotion_text': 'تخفيضات تصل الى 40% على جميع الخدمات لغاية نهاية الشهر',
  'active': 1}]